In [ ]:
# !pip install transformers torch scikit-learn unidecode datasets

In [ ]:
# Importar las librerías
import pandas as pd
import torch
import numpy as np
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report
import matplotlib.pyplot as plt
import seaborn as sns
from torch.optim import AdamW
from transformers import RobertaForSequenceClassification, RobertaTokenizer, AutoTokenizer,  AutoModelForSequenceClassification

In [ ]:
lang = "ukr"
model_name = 'google-bert/bert-base-multilingual-uncased'
max_token_len = 71
num_epochs = 10
lr = 2e-5
save_model = True

In [ ]:
# Leer los archivos de Google Drive
train_path = f'/content/drive/MyDrive/Proyectos/semeval/data/newest/train/{lang}.csv'
val_path = f'/content/drive/MyDrive/Proyectos/semeval/data/newest/dev/{lang}.csv'
test_path = f'/content/drive/MyDrive/Proyectos/semeval/data/newest/dev/{lang}.csv'


In [ ]:
df_train = pd.read_csv(train_path)
df_val = pd.read_csv(val_path)
df_test = pd.read_csv(test_path)

In [ ]:
df_train = pd.concat([df_train, df_val], ignore_index=True)

In [ ]:
col_names = ['anger', 'disgust', 'fear', 'joy', 'sadness', 'surprise']

In [ ]:
for x in col_names:
  df_train[x] = df_train.apply(lambda row: 1 if row[x] > 0 else 0, axis=1)
  df_val[x] = df_val.apply(lambda row: 1 if row[x] > 0 else 0, axis=1)
  df_test[x] = df_val.apply(lambda row: 1 if row[x] > 0 else 0, axis=1)

In [ ]:
# Convertir las columnas de emociones en listas de etiquetas con tipo float32
df_train[col_names] = df_train[col_names].astype('float32')
df_val[col_names] = df_val[col_names].astype('float32')
df_test[col_names] = df_test[col_names].astype('float32')

In [ ]:
# Convertir las columnas de emociones en listas de etiquetas
df_train['labels'] = df_train[col_names].values.tolist()
df_val['labels'] = df_val[ col_names ].values.tolist()
df_test['labels'] = df_test[col_names].values.tolist()


In [ ]:
df_train["labels"] = df_train.apply(lambda x: [float(y) for y in x["labels"]], axis=1)
df_val["labels"] = df_val.apply(lambda x: [float(y) for y in x["labels"]], axis=1)
df_test["labels"] = df_test.apply(lambda x: [float(y) for y in x["labels"]], axis=1)


In [ ]:
df_train

,id,text,anger,disgust,fear,joy,sadness,surprise,labels
0,ukr_train_track_b_00001,Хочу гітару і записувати кавери.,0.0,0.0,0.0,0.0,0.0,0.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
1,ukr_train_track_b_00002,"сенсей зацінив мої поетичні здібності, оце щас...",0.0,0.0,0.0,1.0,0.0,0.0,"[0.0, 0.0, 0.0, 1.0, 0.0, 0.0]"
2,ukr_train_track_b_00003,"мені б снігу трішки на свята для настрою, а та...",0.0,0.0,0.0,0.0,0.0,0.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
3,ukr_train_track_b_00004,Коли ми познайомились і довго після того в теб...,0.0,0.0,0.0,0.0,0.0,0.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
4,ukr_train_track_b_00005,так приємно... не очікувала аж такої реакції)),0.0,0.0,0.0,1.0,0.0,1.0,"[0.0, 0.0, 0.0, 1.0, 0.0, 1.0]"
...,...,...,...,...,...,...,...,...,...
2710,ukr_dev_track_b_00245,а я вночі відбивала напад тараканів...,0.0,1.0,0.0,0.0,0.0,0.0,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0]"
2711,ukr_dev_track_b_00246,страшні речі вчать порядні гарненькі дівчата),0.0,0.0,0.0,1.0,0.0,0.0,"[0.0, 0.0, 0.0, 1.0, 0.0, 0.0]"
2712,ukr_dev_track_b_00247,"Ну я боялась забути щось з того, що брала для ...",0.0,0.0,1.0,0.0,0.0,0.0,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.0]"
2713,ukr_dev_track_b_00248,"Оумайн, coffin bar таки існує!",0.0,0.0,0.0,0.0,0.0,1.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 1.0]"


In [ ]:
# Convertir a Dataset de Hugging Face
train_dataset = Dataset.from_pandas(df_train[['text', 'labels']])
val_dataset = Dataset.from_pandas(df_val[['text', 'labels']])
test_dataset = Dataset.from_pandas(df_test[['text', 'labels']])


In [ ]:
# Tokenizador
tokenizer = AutoTokenizer.from_pretrained(model_name)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.72M [00:00<?, ?B/s]

In [ ]:
# Tokenización
def tokenize_function(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True, max_length=max_token_len)


In [ ]:
# Tokenización
train_dataset = train_dataset.map(tokenize_function, batched=True)
val_dataset = val_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)


Map:   0%|          | 0/2715 [00:00<?, ? examples/s]

Map:   0%|          | 0/249 [00:00<?, ? examples/s]

Map:   0%|          | 0/249 [00:00<?, ? examples/s]

In [ ]:
# Definir el modelo para clasificación multilabel
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=len(col_names), problem_type="multi_label_classification")
model.config.hidden_dropout_prob = 0.3  # Ajustar el dropout al 30%


model.safetensors:   0%|          | 0.00/672M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-multilingual-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Definir métrica de evaluación
def compute_metrics(pred):
    labels = pred.label_ids
    preds = (pred.predictions >= 0.5).astype(int)  # Convertir logits a 0 o 1
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='macro')
    acc = accuracy_score(labels, preds)
    return {'accuracy': acc, 'precision': precision, 'recall': recall, 'f1': f1}


In [ ]:
# Argumentos del entrenamiento
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=num_epochs,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    report_to="none"
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
# Entrenador
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    optimizers=(AdamW(model.parameters(), lr=lr), None)
)

<ipython-input-20-6a4ea6b5ab6f>:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
# Entrenar el modelo
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.274500,0.255741,0.489960,0.055556,0.004065,0.007576
2,0.207900,0.185569,0.642570,0.561512,0.254046,0.321160
3,0.132300,0.129952,0.775100,0.610089,0.455801,0.519464
4,0.137400,0.099304,0.859438,0.634101,0.581200,0.604169
5,0.101400,0.072805,0.895582,0.992063,0.653170,0.711842
6,0.085700,0.057304,0.919679,0.926491,0.769026,0.822562
7,0.070900,0.048270,0.923695,0.940476,0.777720,0.831742
8,0.065000,0.041005,0.935743,0.976190,0.810692,0.863387
9,0.038000,0.036960,0.955823,0.945833,0.902233,0.918634
10,0.039500,0.035630,0.959839,0.957672,0.888344,0.915027


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/m

TrainOutput(global_step=1700, training_loss=0.1256345383735264, metrics={'train_runtime': 268.9636, 'train_samples_per_second': 100.943, 'train_steps_per_second': 6.321, 'total_flos': 990633283475400.0, 'train_loss': 0.1256345383735264, 'epoch': 10.0})

In [ ]:
model.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(105879, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1

### evaluacion

In [ ]:
# Evaluación
from torch.utils.data import DataLoader

def collate_fn(batch):
    input_ids = torch.tensor([item['input_ids'] for item in batch])
    attention_mask = torch.tensor([item['attention_mask'] for item in batch])
    labels = torch.stack([torch.tensor(item['labels']) for item in batch])
    return {'input_ids': input_ids, 'attention_mask': attention_mask, 'labels': labels}

test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False, collate_fn=collate_fn)


In [ ]:
# Realizar inferencias y calcular métricas
all_preds, all_labels = [], []

for batch in test_loader:
    input_ids = batch['input_ids'].to(model.device)
    attention_mask = batch['attention_mask'].to(model.device)
    labels = batch['labels'].to(model.device)

    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)

    logits = outputs.logits
    preds = (torch.sigmoid(logits) > 0.5).int().cpu().numpy()
    labels = labels.cpu().numpy()

    all_preds.extend(preds)
    all_labels.extend(labels)


In [ ]:
# Mostrar métricas
print(classification_report(all_labels, all_preds, target_names=col_names))

              precision    recall  f1-score   support

       anger       0.62      0.67      0.64        12
     disgust       0.64      1.00      0.78         9
        fear       1.00      1.00      1.00        17
         joy       1.00      1.00      1.00        41
     sadness       1.00      0.97      0.99        34
    surprise       1.00      1.00      1.00        19

   micro avg       0.93      0.96      0.94       132
   macro avg       0.88      0.94      0.90       132
weighted avg       0.94      0.96      0.95       132
 samples avg       0.48      0.49      0.48       132



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
# Convertir all_preds y all_labels a arrays de dos dimensiones
all_preds = np.vstack(all_preds)
all_labels = np.vstack(all_labels)

# Calcular accuracy por emoción (por columna)
emotion_accuracy = np.mean(all_preds == all_labels, axis=0)
for i, emotion in enumerate(col_names):
    print(f"Accuracy for {emotion}: {emotion_accuracy[i]:.2f}")

# Calcular accuracy global (exact match ratio)
exact_match_accuracy = np.mean(np.all(all_preds == all_labels, axis=1))
print(f"Exact Match Accuracy: {exact_match_accuracy:.2f}")

# Calcular accuracy promedio por muestra
sample_accuracy = np.mean(all_preds == all_labels)
print(f"Sample Accuracy (Average Accuracy): {sample_accuracy:.2f}")

Accuracy for anger: 0.96
Accuracy for disgust: 0.98
Accuracy for fear: 1.00
Accuracy for joy: 1.00
Accuracy for sadness: 1.00
Accuracy for surprise: 1.00
Exact Match Accuracy: 0.94
Sample Accuracy (Average Accuracy): 0.99


In [ ]:
# Guardar el modelo en Google Drive

output_dir = f'/content/drive/MyDrive/Proyectos/semeval/models/{lang}/multilabel/'

if save_model:
  model.save_pretrained(output_dir)
  tokenizer.save_pretrained(output_dir)
